In [1]:
import requests
import polyline

import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
def osrm_format(coords):
    lat, lon = coords
    return f"{lon},{lat}"

def get_route(url, points):
    points = ';'.join(map(osrm_format, points))
    params = {
        'geometries': 'polyline6',
        'overview': 'full',
    }
    
    response = requests.get(url, params=params)
    routes = response.json()
    
    if routes['code'] != 'Ok':
        print(routes)
        return None
    
    geometry = routes['routes'][0]['geometry']
    return (
        polyline.decode(geometry, 6), 
        routes['routes'][0]['distance'],
    )


In [2]:
start = (20.993686966853396, 105.86939936393794)
stop = (21.043666864560734, 105.83550324106861)